In [1]:
import copy
from random import shuffle,randint

In [2]:
file_name = '20x20.txt'

In [3]:
data, N1,N,M = 0, 0, 0,0
def read_file(file_name):
    global N
    global M 
    global data
    global N1
    file = open(file_name,'r')
    N, M = list(map(int,file.readline().split()))
    data = []
    N1 = 0
    for i in range(N):
        string = [0] * M
        vehicle = list(map(int,file.readline().split()))
        for s in vehicle[1:]:
            string[int(s)-1] = 1
            N1+=1
        data.append(string)

In [4]:
class Cluster(object):
    def __init__(self, vehicles=[], details=[]):
        self._vehicles: list = vehicles
        self._details: list = details
        result = self.get_matrix()
        self.N1_in: int = result[1]
        self.N0_in: int = result[2]
        self.matrix = result[0]
    def __repr__(self):
        s=""
        for m in self.matrix:
            s+= str(m).replace(","," ")[1:-1] +'\n'
        return f"\nCustomer {self._vehicles, self._details, self.N1_in, self.N0_in} \n{s[:-1]}"
    
    def get_matrix(self):
        mat = []
        for v in self._vehicles:
            line = []
            for d in self._details:
                line.append(data[v][d])
            mat.append(line)
            N1_in = sum([sum(m) for m in mat])
            N0_in = len(self._vehicles)*len(self._details) - N1_in
        return mat, N1_in, N0_in
    
    def delete_vehicle(self,number):
        i = self._vehicles.index(number)
        n1 = sum(self.matrix[i][:])
        n0 = len(self._details) - n1
        self.N1_in -= n1
        self.N0_in -= n0
        self.matrix = self.matrix[:i] + self.matrix[i+1:] 
        self._vehicles.pop(i)
    
    def add_vehicle(self,number):
        line = []
        for d in self._details:
            line.append(data[number][d])
        n1 = sum(line)
        n0 = len(self._details) - n1
        self.N1_in += n1
        self.N0_in += n0
        self.matrix.append(line)
        self._vehicles.append(number)
        
    def delete_detail(self,number):
        i = self._details.index(number)
        n1 = sum(r[i] for r in self.matrix)
        n0 = len(self._vehicles) - n1
        self.N1_in -= n1
        self.N0_in -= n0
        [r.pop(i) for r in self.matrix]
        self._details.pop(i)
    
    def add_detail(self,number):
        line = []
        for v in self._vehicles:
            line.append(data[v][number])
        n1 = sum(line)
        n0 = len(self._vehicles) - n1
        self.N1_in += n1
        self.N0_in += n0
        [r.append(line.pop(0)) for r in self.matrix]
        self._details.append(number)

In [5]:
def initial_solution(k=2): 
    best_sum = 0
    K = min(N,M) // k 
    for _ in range(20):
        shuffled_vehicle = list(range(N))
        shuffled_details = list(range(M))
        shuffle(shuffled_vehicle)
        shuffle(shuffled_details)
        solution = []
        for _ in range(K):
            veh, det, i = [], [], 0
            while (i< N//K) and (len(shuffled_vehicle)>1):
                veh.append(shuffled_vehicle.pop(0))
                i+=1
            i=0
            while (i< M//K) and (len(shuffled_details)>1):
                det.append(shuffled_details.pop(0))
                i+=1
            if (len(veh)>0) and (len(det)>0):
                solution.append(Cluster(veh, det))
        solution.append(Cluster(shuffled_vehicle, shuffled_details))
        cur_sum = object_function(solution)
        if (cur_sum > best_sum):
            best_solution = solution
            best_sum = cur_sum
    return best_solution

In [99]:
initial_solution(0)

ZeroDivisionError: integer division or modulo by zero

In [6]:
def is_feasible(solution): #проверка на то, что решение существует
    res = get_output(solution)
    veh = set(res[0])
    det = set(res[1])
    if (len(veh - det)==0) and (len(det - veh)==0):
        return True
    else:
        return veh, det

In [7]:
def object_function(solution):
    n1 = sum(c.N1_in for c in solution)
    n0 = sum(c.N0_in for c in solution)
    return n1 / (N1 + n0)

In [8]:
def get_output(solution, k=0): #преобразование решения в итоговый вариант
    vehicle = ['_'] * N
    details = ['_'] * M
    for i in range(len(solution)):
        for v in solution[i]._vehicles:
            vehicle[v] = i+k
        for d in solution[i]._details:
            details[d] = i+k
    return vehicle, details

In [9]:
get_output(solution,1)

NameError: name 'solution' is not defined

In [9]:
def exchange_vehicle(solution): 
    #функция для LS, позволяющая переместить строку в другой класс (перебирает все строки всех классов)
    best_sum = object_function(solution)
    best_sol = copy.deepcopy(solution)
    for cluster in solution:
        first_cluster = copy.deepcopy(cluster)
        if len(cluster._vehicles)>1:
            for vehicle in cluster._vehicles:
                for c in solution:
                    if (c._vehicles == cluster._vehicles) :
                        continue
                    second_cluster = copy.deepcopy(c)
                    c.add_vehicle(vehicle)
                    cluster.delete_vehicle(vehicle)
                    cur_sum = object_function(solution)
                    if (cur_sum > best_sum):
                        best_sol = copy.deepcopy(solution)
                        best_sum = copy.deepcopy(cur_sum)
                    cluster.add_vehicle(vehicle)
                    c.delete_vehicle(vehicle) 
    solution = (best_sol)
    return copy.deepcopy(solution), best_sum

In [10]:
def exchange_detail(solution):
    #функция для LS, позволяющая переместить столбец в другой класс (перебирает все строки всех классов)
    best_sum = object_function(solution)
    best_sol = copy.deepcopy(solution)
    for cluster in solution:
        first_cluster = copy.deepcopy(cluster)
        if len(cluster._details)>1:
            for detail in cluster._details:
                for c in solution:
                    if (c._vehicles == cluster._vehicles) :
                        continue
                    second_cluster = copy.deepcopy(c)
                    c.add_detail(detail)
                    cluster.delete_detail(detail)
                    cur_sum = object_function(solution)
                    if (cur_sum > best_sum):
                        best_sol = copy.deepcopy(solution)
                        best_sum = copy.deepcopy(cur_sum)
                    cluster.add_detail(detail)
                    c.delete_detail(detail) 
    solution = (best_sol)
    return copy.deepcopy(solution), best_sum

In [11]:
def local_search(solution, vnd):#local search, улучшающий решение
    if vnd == 0:
        vnd = [exchange_vehicle, exchange_detail]
    else:
        vnd = [exchange_detail, exchange_vehicle]
    best_sum = object_function(solution)
#     print("INITIAL ", best_sum)
    old_sum = 0
    step = 0
    while(True):
        while best_sum - old_sum > 0.0001 :
            old_sum = copy.deepcopy(best_sum)
            step += 1
            solution, best_sum = vnd[0](solution)
        old_sum = copy.deepcopy(best_sum)
        solution, best_sum = vnd[1](solution)
        if best_sum - old_sum > 0.0001 :
            continue
        else:
#             print("END AT :{}".format(best_sum))
            break
    return copy.deepcopy(solution), best_sum    

In [12]:
def merge_clusters(cl1,cl2):
    return Cluster(cl1._vehicles + cl2._vehicles, cl1._details + cl2._details)

In [13]:
def split_cluster(cl):
    #предполагаем, что кластер разбивается на два равных по длине кластера
    shuffle(cl._vehicles)
    shuffle(cl._details)
    veh_len = len(cl._vehicles) // 2
    det_len = len(cl._details) // 2
    c1 = Cluster(cl._vehicles[:veh_len], cl._details[:det_len])
    c2 = Cluster(cl._vehicles[veh_len:], cl._details[det_len:])
    return c1,c2

In [14]:
def nice_view(sol,name): #функция для записи ответа в файл
    file = open(name, 'w+')
    res = get_output(sol)
    file.write(str(res[0])[1:-1].replace(', ', ' ') + "\n")
    file.write(str(res[1])[1:-1].replace(', ', ' ') + "\n")
    print(res[0])
    print(res[1])
    file.close()
    total_file = open('VNS_total.txt', 'a+')
    total_file.write(name[:-4] + ' ' + str(object_function(solution))+ "\n")
    print(name[:-4] + ' ' + str(sum([s.object_function() for s in sol]))+ "\n")
    total_file.close()

In [15]:
def merge1(solution, n_shaking_merge):
    if n_shaking_merge == 0:
        return solution
    best_sol = []
    best_sum = -1
    for _ in range(n_shaking_merge):
        temp_sol = copy.deepcopy(solution)
        if len(solution)>1: #слить 2 кластера можно только если их как минимум 2 в решении
            i = randint(0, len(solution)-2)
            j = randint(i+1, len(solution)-1)
            cl = merge_clusters(temp_sol.pop(i), temp_sol.pop(j-1))
            temp_sol.append(cl)
            if object_function(temp_sol) > best_sum:
                best_sol = copy.deepcopy(temp_sol)
                best_sum = object_function(temp_sol)
    return best_sol

In [16]:
def merge(solution):
    if len(solution)>1: #слить 2 кластера можно только если их как минимум 2 в решении
        i = randint(0, len(solution)-2)
        j = randint(i+1, len(solution)-1)
        cl = merge_clusters(solution.pop(i), solution.pop(j-1))
        solution.append(cl)
    return solution

In [17]:
def split(solution):
    i = randint(0, len(solution)-1)
    k = 0
    while (len(solution[i]._vehicles)<2 or len(solution[i]._details)<2): 
        #разбить кластер можно только если в нем больше 2 строк и 2 столбцов
        i = randint(0, len(solution)-1)
        k +=1
        if k > len(solution):
            break
    else:
        cls = split_cluster(solution.pop(i))
        solution += cls
    return solution

In [18]:
def split1(solution, n_shaking_split):
    if n_shaking_split == 0:
        return solution
    best_sol = []
    best_sum = -1
    for _ in range(n_shaking_split):
        temp_sol = copy.deepcopy(solution)
        i = randint(0, len(solution)-1)
        k = 0
        while (len(solution[i]._vehicles)<2 or len(solution[i]._details)<2): 
            #разбить кластер можно только если в нем больше 2 строк и 2 столбцов
            i = randint(0, len(solution)-1)
            k +=1
            if k > len(solution):
                break
        else:
            cls = split_cluster(temp_sol.pop(i))
            temp_sol += cls
            if object_function(temp_sol) > best_sum:
                best_sol = copy.deepcopy(temp_sol)
                best_sum = object_function(temp_sol)
    return best_sol

In [38]:
#MAIN FUNCTION
#вся магия здесь
def main(elem_in_cluster, repeat, vnd, shaking, n_shaking, n_best_sh):
    read_file(file_name)
    if shaking == 0:
        shaking = [merge1, split1, n_best_sh]
    else:
        shaking = [split1, merge1, n_best_sh]
    BS = 0
    BSOL = []
    for _ in range(repeat):
        solution = initial_solution(elem_in_cluster)
        solution, best_sum = copy.deepcopy(local_search(solution, vnd))  
        best_sol = copy.deepcopy(solution)
        for sh_oper in range(2):
            shaking_counter = 0
            while shaking_counter < n_shaking:
                solution = copy.deepcopy(best_sol)
                solution = copy.deepcopy(shaking[sh_oper](solution, shaking[-1]))
                solution, cur_sum = copy.deepcopy(local_search(solution, vnd))  
                if cur_sum > best_sum:
#                     print("Improved in ", shaking[sh_oper].__name__, " to ", cur_sum, ' at ', shaking_counter)
                    shaking_counter = 0
                    best_sum = copy.deepcopy(cur_sum)
                    best_sol = copy.deepcopy(solution)
                else:
                    shaking_counter += 1
        print(best_sum)           
        if best_sum > BS:
            BS = best_sum
            BSOL = best_sol
    return BSOL, BS

In [40]:
main(5, 10, 0,0, 10, 3)

0.42028985507246375
0.41353383458646614
0.40625
0.4305555555555556
0.4195804195804196
0.39263803680981596
0.4126984126984127
0.40816326530612246
0.40601503759398494
0.39072847682119205


([
  Customer ([3, 6, 17, 8, 5, 0], [8, 6, 0, 5, 15], 19, 11) 
  1  0  0  1  0
  1  1  1  1  1
  0  1  1  0  1
  1  0  0  1  1
  0  1  0  1  0
  1  0  1  1  1, 
  Customer ([10, 2, 9, 11, 7, 13, 16], [3, 14, 13, 17, 18, 12, 10], 30, 19) 
  1  1  1  1  1  1  0
  1  0  1  1  0  1  1
  0  1  0  0  1  1  1
  1  0  1  1  0  0  1
  1  1  1  1  0  0  0
  0  0  1  1  1  1  0
  1  1  1  0  0  0  0, 
  Customer ([18], [11], 0, 1) 
  0, 
  Customer ([12, 1], [16, 1, 9, 19], 7, 1) 
  1  1  1  0
  1  1  1  1, 
  Customer ([15], [4], 1, 0) 
  1, 
  Customer ([14, 19, 4], [2, 7], 5, 1) 
  1  1
  1  1
  1  0], 0.4305555555555556)

In [37]:
solution = initial_solution(2)
solution, cur_sum = copy.deepcopy(local_search(solution,0))  
print( cur_sum)
solution, cur_sum = copy.deepcopy(local_search(solution, 0))  
print(cur_sum)
solution, cur_sum = copy.deepcopy(local_search(solution, 0))  
print(cur_sum)
solution, cur_sum = copy.deepcopy(local_search(solution, 0))  
print(cur_sum)
# res = main(5, 30, 0,0, 3,3)


0.3247863247863248
0.3247863247863248
0.3247863247863248
0.3247863247863248


In [97]:
is_feasible(res[0])

True

In [40]:
best_sol
s = 0
for c in best_sol:
    matrix = c.get_matrix()[0]
    n1 = sum([sum(m) for m in c.matrix])
    n0 = len(c.matrix) * len(c.matrix[0]) - n1
    s+= n1/(N1+n0)

In [41]:
s

0.49962013069407435

In [45]:
a = sum([sum(m) for m in data])

In [46]:
a, N1

(111, 111)

In [80]:
best_sol
s = 0
n1 = 0
n0 = 0
for c in best_sol:
    veh = c._vehicles
    det = c._details
    rn1 = sum([sum([data[v][d] for d in det]) for v in veh])
    rn0 = len(veh) * len(det) - rn1
    n1+=rn1
    n0+=rn0
s+= n1/(N1+n0)

In [81]:
s

0.38961038961038963

In [52]:
c = best_sol[0]
veh = c._vehicles
det = c._details
n1 = sum([sum([data[v][d] for d in det]) for v in veh])
n0 = len(veh) * len(det) - n1
n0,n1

(31, 32)

In [82]:
best_sol

[
 Customer ([15], [4, 11], 1, 1) 
 1  0, 
 Customer ([0, 17], [0], 2, 0) 
 1
 1, 
 Customer ([5, 16, 13, 2, 8, 7, 10, 6, 11], [5, 17, 3, 18, 6, 13, 15], 35, 28) 
 1  0  0  0  1  1  0
 0  0  1  0  1  1  1
 0  1  0  1  0  1  0
 0  1  1  0  0  1  1
 1  0  0  1  0  1  1
 1  1  1  0  0  1  0
 0  1  1  1  0  1  0
 1  1  0  1  1  1  1
 0  1  1  0  0  1  0, 
 Customer ([1, 9, 12], [16, 12, 9, 1, 10, 14, 19], 15, 6) 
 1  1  1  1  0  1  1
 0  1  1  0  1  1  0
 1  1  1  1  1  0  0, 
 Customer ([19, 18, 14, 4, 3], [7, 8, 2], 7, 8) 
 1  0  1
 0  0  0
 1  1  1
 0  0  1
 0  1  0]

In [47]:
best_sol

[
 Customer ([8, 6, 3, 16, 17, 14, 0, 5, 15], [15, 0, 8, 6, 5, 4, 13], 32, 31, 0.22535211267605634) 
 1  0  1  0  1  0  1
 1  1  1  1  1  0  1
 0  0  1  0  1  0  0
 1  0  0  1  0  1  1
 1  1  0  1  0  0  0
 1  0  1  0  0  1  0
 1  1  1  0  1  0  0
 0  0  0  1  1  0  1
 1  0  0  0  0  1  1, 
 Customer ([12, 2, 7, 4, 9, 11, 1, 13, 10], [17, 19, 10, 12, 9, 16, 2, 3, 14, 1], 41, 49, 0.25625) 
 1  0  1  1  1  1  0  0  0  1
 1  0  1  1  0  0  1  1  0  0
 1  1  0  0  1  0  0  1  1  0
 0  0  0  1  0  0  1  0  1  0
 0  0  1  1  1  0  0  0  1  0
 1  0  1  0  0  1  0  1  0  0
 0  1  0  1  1  1  1  0  1  1
 1  0  0  1  0  1  0  0  0  0
 1  0  0  1  0  0  0  1  1  0, 
 Customer ([18], [11], 0, 1, 0.0) 
 0, 
 Customer ([19], [7, 18], 2, 0, 0.018018018018018018) 
 1  1]

In [54]:
solution_matrix = [['_']*(M+1) for _ in range(N+1)]
for i in range(len(solution)):
    for v in solution[i]._vehicles:
        for d in solution[i]._details:
            solution_matrix[v][d] = i + 1
print_matrix(solution_matrix)


_ _ _ _ _ _ _ _ 6 _ _ _ 6 6 _ _ 6 _ _ _ _ _ _ _ _ _ _ _ 6 _ 6 _ 6 _ _ _ _ _ _ 6 _ 
_ _ _ _ _ _ _ _ 6 _ _ _ 6 6 _ _ 6 _ _ _ _ _ _ _ _ _ _ _ 6 _ 6 _ 6 _ _ _ _ _ _ 6 _ 
_ 3 _ _ _ _ _ _ _ _ _ 3 _ _ 3 3 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 3 _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ 5 _ _ 5 _ _ _ _ _ _ _ _ _ 5 _ _ _ 5 _ _ 5 _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ 7 _ _ _ _ _ _ _ _ _ _ _ 7 7 _ _ _ _ _ _ _ _ _ _ _ 7 _ _ _ _ _ _ 
_ _ _ 1 1 _ _ _ _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ 1 1 _ _ 1 _ _ _ _ _ _ _ 1 _ _ _ 
_ _ _ _ _ _ _ 5 _ _ 5 _ _ _ _ _ _ _ _ _ 5 _ _ _ 5 _ _ 5 _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ 1 1 _ _ _ _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ 1 1 _ _ 1 _ _ _ _ _ _ _ 1 _ _ _ 
_ _ _ _ _ _ _ _ 6 _ _ _ 6 6 _ _ 6 _ _ _ _ _ _ _ _ _ _ _ 6 _ 6 _ 6 _ _ _ _ _ _ 6 _ 
_ _ _ _ _ 4 _ _ _ _ _ _ _ _ _ _ _ _ _ 4 _ _ _ 4 _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
_ _ _ _ _ _ _ _ _ 7 _ _ _ _ _ _ _ _ _ _ _ 7 7 _ _ _ _ _ _ _ _ _ _ _ 7 _ _ _ _ _ _ 
_ _ _ 1 1 _ _ _ _ _ _ _ _ _ _ _ _ 1 _ _ _ _ _ _ _ 1 1 _ _ 1 _ _ _ _ _ _ _ 1 _ _ _ 
_ 3 

In [37]:
c = best_sol[0]

In [45]:
sum([sum(m) for m in c.matrix])

20

In [487]:
BSOL = best_sol
BSUM = best_sum

In [488]:
is_feasible(BSOL)

True

In [489]:
get_output(BSOL)

([1, 6, 0, 8, 4, 3, 7, 3, 4, 5, 0, 5, 4, 8, 2, 2, 6, 5, 2, 7, 1, 3, 4, 7],
 [1,
  7,
  4,
  5,
  3,
  4,
  6,
  8,
  1,
  0,
  8,
  4,
  6,
  2,
  0,
  3,
  1,
  5,
  1,
  5,
  8,
  6,
  0,
  5,
  7,
  2,
  2,
  8,
  6,
  3,
  6,
  7,
  1,
  3,
  4,
  2,
  2,
  3,
  3,
  6])

In [478]:
nice_view(BSOL, '24x40.sol')

[1, 0, 1, 2, 0, 1, 1, 2, 1, 1, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 1, 0, 1, 2, 1, 2, 0, 2, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
37x53 0.6289532711953234

